In [184]:
import numpy as np
import pandas as pd
import os
import re

定义初始变量
======

In [185]:
df_raw = pd.DataFrame()#全局记录nmr数据
df_sample = pd.DataFrame(['number','name','sample','temperature'])#全局记录样品信息
df_sample
path_tp = 'a'#全局工作地址

定义预设函数
======

1.展示文件夹内文件目录

In [186]:
def showDir (FileList):
    for index, name in enumerate(FileList)  :
        if index <50:
            print(index, name)
        else: break
    return 0

2.检查文件列表中是否已存在某文件

In [187]:
def checkFile(FileName,FileList) :
    key = 0
    for str in FileList:
        if str == FileName:
            key = 1
            break
    return key

In [188]:
def chooseFile (path,message):
    print('Now in ' + path)
    str_tp = ''
    showDir(path)
    str_tp = input(message)
    

3.打开旧文件

In [189]:
def loadwork (df_raw,df_sample):
    str_tp = ''
    #建立一个临时字符串存放地址
    str_tp = input('Read old work?(y/n)')
    if str_tp=='y':
        mainPath = r'C:\Users\ikeno\Desktop\Pojects'
        path_tp=os.path.join(mainPath,'log')
        fileList = os.listdir(path_tp)
        showDir(fileList)
        str_tp = input('Choose a work log by index number')
        try:
            path_tp = os.path.join(path_tp,fileList[int(str_tp)])
            df_raw = pd.read_csv(os.path.join(path_tp,'Data.csv'),index_col=0,header =None)
            df_sample = pd.read_csv(os.path.join(path_tp,'Sample.csv'),index_col=0)
        except:
            path_tp  = os.path.join(path_tp,str_tp)
            df_raw = pd.read_csv(os.path.join(path_tp ,'Data.csv'),index_col=0,header =None)
            df_sample = pd.read_csv(os.path.join(path_tp ,'Sample.csv'),index_col=0)
        finally:
            df_raw=df_raw.T.reset_index(drop=True).iloc[:,1:]
            df_raw=df_raw.T.reset_index(drop=True)
            print('Load success')
    return df_raw,df_sample
    

4.选择文件夹

In [190]:
def openFile(path1,path2):
    path_tp= ''
    str_tp =''
    mainPath = r'C:\Users\ikeno\Desktop\Pojects'
    try:
        path_tp= path1
    except:
        path_tp= mainPath
    print('Now in ' + path_tp)
    fileList = os.listdir(path_tp)
    showDir(fileList)
    str_tp = input('Input document(by index) to open:\n'
                   'main'': back to main\n'
                   'return'': return to one level up\n'
                   'ready'': use path showing\n'
                   'skip'':skip this step')
    if str_tp == 'main':
        path_tp = mainPath
        return openFile(mainPath,path2)
    elif str_tp == 'return':
        return openFile(path2,path_tp)
    elif str_tp =='ready':
        path_tp=os.path.abspath(path_tp)
        print('Choose ' + path_tp)
        return str(path_tp)
    elif str_tp =='skip':
        print('Please input a dirctroy for work path')
        return 'a'
    else :
        try:
            path_tp= os.path.join(path_tp,fileList[int(str_tp)])
            return openFile(path_tp,path1)
        except os.path.join(path_tp,str_tp):
            path_tp= os.path.join(path_tp,str_tp)
            return openFile(path_tp,path1)
        except FileNotFoundError:
            return openFile(path_tp,path2)




5.提取数据

In [191]:
def openFile_data(newPath):
    fileList =os.listdir(newPath)
    df_raw_tp = pd.DataFrame()
    for fileName in fileList :
        if re.search(r'.dps',fileName) :
            str_tp = fileName
            try:
                filePath= os.path.join(newPath,str_tp)#要改为确认文件
            except FileNotFoundError:
                continue
            
            df_1 = pd.read_csv(filePath,sep="\t",header=None)
            df_0 = pd.DataFrame(['0','time',filePath]).T
            df_1 = pd.concat([df_0,df_1],axis=0,join='outer').iloc[:,1:]
            df_1 = df_1.reset_index(drop=True)
            df_1=df_1.T
            if df_raw_tp.shape == (0, 0):
                df_raw_tp = df_1
            else:
                df_2=pd.DataFrame(df_1.copy().loc[2]).T
                df_raw_tp = df_raw_tp.reset_index(drop= True)
                df_raw_tp=pd.concat([df_raw_tp,df_2],axis=0,join='outer',ignore_index=True)
            
    return df_raw_tp

In [192]:
def openFile_sample(newPath):
    fileList =os.listdir(newPath)
    df_sample = pd.DataFrame(['number','name','sample','temperature'])
    for fileName in fileList :
        if re.search(r'.dps',fileName) :
            str_tp = fileName
            try:
                filePath= os.path.join(newPath,str_tp)
            except FileNotFoundError:
                continue
            df_temp = pd.DataFrame(index=range(4),columns=range(1))
            #记录文件名
            df_temp.iloc[1]=filePath
            #记录样品名
            name_pattern = re.search(r'^(.+)(?=-)',str_tp)
            try:
                df_temp.iloc[2]=name_pattern.group()
            except:
                df_temp.iloc[2]='NaN'
            #记录温度
            temp_pattern = re.search(r'(\d+)K',newPath)
            #temp_pattern = re.search(r'(\d+)K',str_tp)
            try:
                df_temp.iloc[3]=temp_pattern.group()
            except:
                df_temp.iloc[2]='NaN'

            #保存到df_sample
            if checkFile(fileName,df_sample.iloc[1,:])==0:
                df_sample = pd.concat([df_sample,df_temp],axis=1,join='outer',ignore_index=True)
            else:
                continue
            
    return df_sample

6.连续读取同一文件夹内的子文件夹

In [193]:
def Routine(workPath,df_raw_tp,df_sample):
    str_tp=''
    path_tp=workPath
    print('Now in ' + path_tp)
    fileList = os.listdir(workPath)
    showDir(fileList)
    str_tp=input('choose a document(by index):\nor input [pass] to pass')
    print(str_tp)
    if str_tp != 'pass':
        path_tp= os.path.join(path_tp,fileList[int(str_tp)])
        df_work_data = openFile_data(path_tp)
        df_raw_tp=pd.concat([df_raw_tp,df_work_data],axis=0,join='outer',ignore_index=True)
        df_work_sample = openFile_sample(path_tp).drop(columns=0)
        df_sample = pd.concat([df_sample,df_work_sample],axis=1,join='outer',ignore_index=True)
    
    str_tp=input('go on?[y/n]:\ny: open next document\nall: do all\nn: finish')
    
    if str_tp=='all':
        for file in fileList:
            path_tp= os.path.join(workPath,file)
            df_work_data = openFile_data(path_tp).drop(index=0)
            df_raw_tp=pd.concat([df_raw_tp,df_work_data],axis=0,join='outer',ignore_index=True)
            df_work_sample = openFile_sample(path_tp).drop(columns=0)
            df_sample = pd.concat([df_sample,df_work_sample],axis=1,join='outer',ignore_index=True)
        print(df_sample)
        return df_raw_tp,df_sample
    elif str_tp=='n':
        return df_raw_tp,df_sample
    elif str_tp=='y':
        return Routine(workPath,df_raw_tp,df_sample)
    else:
        print('Fail by mistake')
    


读取工作表
======

In [194]:
df_raw,df_sample = loadwork(df_raw,df_sample)
mainPath = r'C:\Users\ikeno\Desktop\Pojects'


选择工作文件夹
======

In [195]:
path_tp=openFile(mainPath,mainPath)
print(path_tp)

Now in C:\Users\ikeno\Desktop\Pojects
0 DailyPot_20240502
1 DataProcessing
2 DataProcessing_test
3 DataToCheck
4 Guanidinium_phosphate
5 log
6 poject_Panel
7 PolyPropylene
8 新規 Microsoft Excel ワークシート.xlsx
Now in C:\Users\ikeno\Desktop\Pojects\Guanidinium_phosphate
0 Data
1 Methods
2 phosphoric_acid_mseOnly_20240424.tbl
Now in C:\Users\ikeno\Desktop\Pojects\Guanidinium_phosphate\Data
0 273K
1 283K
2 298K
3 308K
4 318K
5 328K
6 338K
7 348K
8 358K
9 368K
10 378K
Choose C:\Users\ikeno\Desktop\Pojects\Guanidinium_phosphate\Data
C:\Users\ikeno\Desktop\Pojects\Guanidinium_phosphate\Data


正式读取（可直接重新操作部分）

In [196]:
if path_tp=='a':
    workPath=input('Input work path')
else:
    workPath=path_tp
df_work_data=df_raw
df_work_sample=df_sample
df_work_data,df_work_sample = Routine(workPath,df_work_data,df_work_sample)
print(df_work_data)
print()
print(df_work_sample)

Now in C:\Users\ikeno\Desktop\Pojects\Guanidinium_phosphate\Data
0 273K
1 283K
2 298K
3 308K
4 318K
5 328K
6 338K
7 348K
8 358K
9 368K
10 378K
pass
            0               1            2            3            4   \
0       number             NaN          NaN          NaN          NaN   
1         name  CHS_T1-mse.dps  No1-mse.dps  No2-mse.dps  No3-mse.dps   
2       sample          CHS_T1          No1          No2          No3   
3  temperature            273K         273K         273K         273K   

           5               6            7            8            9   ...  \
0         NaN             NaN          NaN          NaN          NaN  ...   
1  S7-mse.dps  CHS_T1-mse.dps  No1-mse.dps  No2-mse.dps  No3-mse.dps  ...   
2          S7          CHS_T1          No1          No2          No3  ...   
3        273K            283K         283K         283K         283K  ...   

               46           47           48           49          50  \
0             NaN          N

In [197]:
print(df_work_data)
print(df_work_sample)


               0          1          2          3          4          5    \
0   CHS_T1-mse.dps   38.25852  36.827535  35.995928  35.389724  34.756654   
1      No1-mse.dps  22.471899  21.246499   20.72383  19.558207  18.333394   
2      No2-mse.dps  22.395489  20.572078  19.689884  18.654358  17.421083   
3      No3-mse.dps  53.753702  52.341335   51.90687  51.333499   50.36956   
4       S7-mse.dps  39.597881  38.202376   37.47381  36.870644  36.200847   
5   CHS_T1-mse.dps  62.328069  60.775334  59.708556   58.67231  57.764741   
6      No1-mse.dps  20.870193  19.267243  18.586689  18.035643   17.37408   
7      No2-mse.dps  22.852492  21.156511  20.181672  19.112669  18.020468   
8      No3-mse.dps  39.801047  38.776973  38.258201  37.845859  37.510351   
9       S7-mse.dps  62.575256   60.89912  59.862629  58.961099  58.176055   
10  CHS_T1-mse.dps  64.139784  62.354074  61.206378  60.383524  59.649103   
11     No1-mse.dps  20.998432  19.543374  18.693723  17.748387  16.943382   

保存！
======

In [198]:
df_work_sample =df_work_sample.T



修改表头等格式

In [200]:
df_tp = df_work_sample
df_tp_key = df_tp.values.tolist()[0]
df_tp.drop([0],inplace=True)
df_tp.columns = df_tp_key
df_tp

,number,name,sample,temperature
1,NaN,CHS_T1-mse.dps,CHS_T1,273K
2,NaN,No1-mse.dps,No1,273K
3,NaN,No2-mse.dps,No2,273K
4,NaN,No3-mse.dps,No3,273K
5,NaN,S7-mse.dps,S7,273K
6,NaN,CHS_T1-mse.dps,CHS_T1,283K
7,NaN,No1-mse.dps,No1,283K
8,NaN,No2-mse.dps,No2,283K
9,NaN,No3-mse.dps,No3,283K
10,NaN,S7-mse.dps,S7,283K
